<a href="https://colab.research.google.com/github/JoongseokPark/machine_learning_2023/blob/main/%EA%B8%B0%EA%B3%84%ED%95%99%EC%8A%B5_8%EC%A3%BC%EC%B0%A8_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_hub as tfhub

#객체 탐색 

In [ ]:
img = '/content/city.jpg'
img = tf.io.read_file(img) #파일을 string으로 변환
img = tf.image.decode_jpeg(img,channels=3) #문자를 숫자 텐서로 변환
img = tf.image.convert_image_dtype(img,tf.float32) #0~1로 정규화

plt.figure(figsize=(15,10))
plt.imshow(img)

In [ ]:
img.shape

In [ ]:
img_input = tf.expand_dims(img,0) #batch_size 추가, 4차원 텐서로 입력 
img_input.shape

In [ ]:
#Tensorflow hub에서 모델 가져오기 - FasterRCNN + InceptionResNet V2
model = tfhub.load("https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1")

In [ ]:
model.signatures.keys()

In [ ]:
#객체탐지 모델 생성
obj_detector = model.signatures['default']
obj_detector

In [ ]:
#모델 사용해서 예측
result = obj_detector(img_input)
result.keys()

In [ ]:
#탐지한 객체의 개수
len(result["detection_scores"])

In [ ]:
boxes = result["detection_boxes"] 
labels = result["detection_class_entities"]
scores = result['detection_scores']
names = result['detection_class_names']

In [ ]:
img_height,img_width = img.shape[0],img.shape[1]
obj_to_detect = 10
for i in range(min(obj_to_detect, boxes.shape[0])):
  if scores[i] >= 0.2:
    (ymax,xmin,ymin,xmax) = (boxes[i][0]*img_height,boxes[i][1]*img_width,
                             boxes[i][2]*img_height,boxes[i][3]*img_width)
  print("{} : {}, {}, {}, {}".format(scores[i],ymax,xmin,ymin,xmax))

##탐지 결과 시각화 

In [ ]:
boxes = result["detection_boxes"] 
labels = result["detection_class_entities"]
scores = result['detection_scores']

#image size
img_height,img_width = img.shape[0],img.shape[1]

#max number of object detection
obj_to_detect = 10

for i in range(min(obj_to_detect, boxes.shape[0])):
  if scores[i] >= 0.2:
    (ymax,xmin,ymin,xmax) = (boxes[i][0]*img_height,boxes[i][1]*img_width,
                             boxes[i][2]*img_height,boxes[i][3]*img_width)
    
    plt.imshow(img)
    plt.plot([xmin,xmax,xmax,xmin,xmin],[ymin,ymin,ymax,ymax,ymin],
             color='yellow',linewidth=2)
    class_name = labels[i].numpy().decode('utf-8')
    infer_score = int(scores[i].numpy()*100)
    annotation = "{} : {}%".format(class_name,infer_score)
    plt.text(xmin+10,ymax-10,annotation,color='white',backgroundcolor='blue',fontsize=5)